In [14]:
from vectorgebra import *
# VERSION 2.7.3

## Linear Algebra on Vectorgebra

All sorts of linear algebra and related operations are defined on Vectorgebra, and mostly on vectorgebra.Matrix. These range from basic determinant, inverse, etc. operations to equation solving algorithms, eigenvalue related and decomposition related operations.

There are many many methods defined for those, we are not going to cover all of those. But we will cover the methods that are probably going to be used most.

We can start by the most basic 3 operations; determinant, inverse, transpose.

In [15]:
m = Matrix.randMint(3, 3, 0, 5, False)
print(f"Original matrix:\n{m}\n")
print(f"Determinant of the matrix: {m.determinant()}\n")
m_inverse = m.inverse(decimal=False)
print(f"Inverse of the matrix:\n{m_inverse}\n")
print(f"Check if the inverse is correct:\n{m * m_inverse}\n") # Check out the %error.
# Output of above should be an identity matrix since A * A-1 = I

print(f"Transpose of the matrix:\n{m.transpose()}\n")

Original matrix:
[ 2, 0, 1]
[ 3, 3, 3]
[ 2, 5, 4]

Determinant of the matrix: 3.0

Inverse of the matrix:
[ -0.04827072023700316, 0.5870669852426267  , -0.3771223564289387 ]
[ -0.6782925149779264 , 0.5007133750041447  , -0.13498296073669735]
[ 0.8280752864829879  , -0.8695978166810783 , 0.5785418434261905  ]

Check if the inverse is correct:
[ 0.7315338460089815 , 0.3045361538041751 , -0.1757028694316869]
[ 0.3045361538041753 , 0.654547630697079  , 0.19930957878166344]
[ -0.1757028694316869, 0.1993095787816639 , 0.8850078571633979 ]

Transpose of the matrix:
[ 2, 3, 2]
[ 0, 3, 5]
[ 1, 3, 4]


### Inversion

You wil see that inverse is not quite correct. vectorgebra.Matrix.inverse() method is a sophisticated method that requires multiple specific arguments to be used correctly. decimal.Decimal choice that is written in the above code is one of them. 

Other than that, firstly, there are 4 different inversion methods in vectorgebra. Those are given to the function via "method" argument. Possible choices are "analytic", "iterative", "gauss", "neumann". Iterative solution is the default method that uses the Newton method. To use it with higher accuracy, you need to probably increase the "resolution" arguments value. This defines the number of iterations that are made during the calculation of the Newton method.

Analytic methods use is discouraged for higher dimensional matrices than 5. It is however the fastest method for very low dimensional matrices. This method uses the coefficient method to calculate the adjoint matrix and then the inverse. The result of this method is _always_ exact and true (Except maybe for some very ill conditioned matrices that would have values that are very very close to zero. And even then just set "decimal=True".). 

The Neumann method only works for properly conditioned matrices. If you work on very specific cases you can use it and it is fast at its job. For more information, see the [github](https://github.com/ahmeterdem1/Vector) page.

The "gauss" method is the Gauss-Jordan elimination method, and it is the fastest among all. It should be used for very high dimensional matrices as it is very effective at that case. This method accepts parameters "lowlimit" and "highlimit". You may need to tune these values to manage the accuracy of the gauss method. This method is heavily dependent on divisions and because of that there may be floating point errors. Even decimal.Decimal class is sometimes unable to overcome those errors. In that case, you lower the "highlimit" and increase the "lowlimit". Divisions that result in values smaller than "lowlimit" are assumed zero, divisions that result in values higher than "highlimit" are also assumed zero. This is because when the denominator is too small but not zero, values blow up. "highlimit" literally limits those errors. Even though all the required tuning (default values will work just fine for most of the cases), this method is blazing fast and works very efficiently. And also the results are very very accurate. 

### Determinant

Determinant also can be done via 2 different methods. One of these is "analytic" which is the basis of the analytic inversion method. This method is very slow for more than 5 dimensional matrices, however the value is always exact. The other method is "echelon", which is the default method. This method first reduces the matrix into its echelon form, then multiplies the diagonals since it would be a triangular matrix. This method is very fast and should be used for most of the cases. For 2, 3 and 4 dimensional matrices you may opt to use the analytic method for better accuracy (and it is faster for those matrices).

### Least Squares

There are different methods to solve equations or find the best fitting result in vectorgebra.Matrix. We will only look at the least squares method for a system of equations. Let's first create a randomized dataset to work on.

In [16]:
line = lambda x: 2*x + 3 # We will generate points around this line

x_list = [k for k in range(10)]
y_list = [line(x + random.uniform(-0.5, 0.5)) for x in x_list] #We will deviate from the line randomly.
v_list = [Vector(1, x) for x in x_list]

y = Vector(*y_list)
A = Matrix(*v_list)

result = A.least_squares(y, resolution=40, decimal=False) # Solve for Ax=y

print(f"The resultant vector is: {result}")
print(f"The error vector is: {(Vector(3, 2) - result).map(abs)}") # "abs" is defined in vectorgebra

The resultant vector is: [2.822594475427586, 2.045436408073498]
The error vector is: [0.1774055245724142, 0.045436408073498136]


Least squares method utilizes inverses of matrices. Every argument that vectorgebra.Matrix.inverse() has is also accepted by vectorgebra.Matrix.least_squares(). The method returns a vector that has as values $b_{0}$ and $b_{1}$ in order.

Least squares is a good example for entry to production code usages in vectorgebra. Above example has a little bit of everything. We start by defining an objective, then we generate our simulation data. After that we structure our simulation data into vectors and matrices and then we finally utilize .least_squares() to get to a solution.

At the end we finish by the calculation of the error vector.

Let's look at one last method. THe echelon form. With this method, I guess we have completed the top 5 of linear algebra. This method only accepts square matrices. To row reduce non-square matrices, you may append all zero vectors at the end of the matrix until you reach a square matrix.

In [17]:
m = Matrix.randMint(3, 3, 0, 5, False)
print(f"Original matrix:\n{m}\n")
m_reduce = m.echelon()
print(f"Reduced matrix:\n{m_reduce}\n")

print("Compare the determinants of both (via analytic method to not cheat):\n")
print(f"Original determinant:{m.determinant(choice='analytic')}")
print(f"Determinant after reduction:{m_reduce.determinant(choice='analytic')}")

Original matrix:
[ 4, 4, 2]
[ 5, 5, 1]
[ 2, 3, 3]

Reduced matrix:
[ 4.0 , 3.0 , 0.0 ]
[ 0.0 , 0.75, 0.0 ]
[ 0.0 , 1.0 , 2.0 ]

Compare the determinants of both (via analytic method to not cheat):

Original determinant:6
Determinant after reduction:6.0


Always keep in mind that row reduction based operations like echelon form, echelon determinant, Gauss-Jordan inversion, etc. are prone to floating point errors. However, these errors are rare and the related algorithms are probably the fastest of the library.